In [184]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # this is used for the plot the graph 
from datetime import date
import random

from sklearn.decomposition import PCA
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE

import torch
from torch import nn
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import TensorDataset, DataLoader

import keras
from keras.models import Sequential
from keras.layers import Dense

%matplotlib inline

import random 
random.seed(42)

In [185]:
csv_path_1 = "training_dataset.csv"
csv_path_2 = "score.csv"

# 0. First, let's understand our data...

In [186]:
df = pd.read_csv(csv_path_1) # load the pandas dataframe
df_score = pd.read_csv(csv_path_2)
initial_cols_to_drop = ["Unnamed: 0","Unnamed: 0.1", "period", "test", "recent_date", "date"] 
for col_name in initial_cols_to_drop: # drops columns that aren't supposed to be in dataset
    try:
        df = df.drop(columns=[col_name])
        df_score = df_score.drop(columns=[col_name])
    except:
        continue
#df = df.rename(columns={"Unnamed: 0.1": "TODO_FIND_COLUMN_NAME_2"})
display(df.head())

,dt,weekday,year,id_driver,id_carrier_number,dim_carrier_type,dim_carrier_company_name,home_base_city,home_base_state,carrier_trucks,...,most_recent_load_date,load_day,loads,marketplace_loads_otr,marketplace_loads_atlas,marketplace_loads,brokerage_loads_otr,brokerage_loads_atlas,brokerage_loads,total_loads
0,2019-12-16,Monday,2019,21350,U0109015,Owner Operator,CA&F TRUCKING,Maywood,CA,"[""poweronly""]",...,2021-02-17,2019-12-16,2,0,438,438,0,45,45,483
1,2021-01-15,Friday,2021,36437,C0097727,Fleet,New opportunities inc,Los Angeles,CA,"[""poweronly"", ""boxtruck""]",...,2021-02-03,2021-01-15,1,2,72,74,0,1,1,75
2,2019-12-26,Thursday,2019,19323,U0107081,Owner Operator,RAS,Compton,CA,"[""poweronly""]",...,2020-09-25,2019-12-26,1,0,180,180,0,2,2,182
3,2021-02-10,Wednesday,2021,34809,C0094651,Fleet,NFS asset Drayage,Lynwood,CA,"[""poweronly"", ""dryvan""]",...,2021-02-17,2021-02-10,3,0,0,0,0,0,0,62
4,2017-07-24,Monday,2017,4728,U0094376,Owner Operator,joes transportation,Norco,CA,"[""dryvan""]",...,2017-10-11,2017-07-24,2,57,0,57,314,0,314,371


In [187]:
df.describe()

,year,id_driver,num_trucks,days_signup_to_approval,loads,marketplace_loads_otr,marketplace_loads_atlas,marketplace_loads,brokerage_loads_otr,brokerage_loads_atlas,brokerage_loads,total_loads
count,83414.000000,83414.000000,83344.000000,71124.000000,83414.000000,83414.000000,83414.000000,83414.000000,83414.000000,83414.000000,83414.000000,83414.000000
mean,2018.960930,18222.414954,22.582921,298.752489,2.076270,29.477762,71.579675,101.057436,148.258422,13.073021,161.331443,266.502661
std,1.359343,11667.704926,48.829719,390.345107,2.672163,88.171940,194.532776,214.502147,415.978060,42.241592,413.792137,448.806175
min,2015.000000,20.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,2018.000000,7890.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,37.000000
50%,2019.000000,16299.000000,4.000000,61.000000,1.000000,2.000000,0.000000,13.000000,15.000000,0.000000,37.000000,110.000000
75%,2020.000000,28974.000000,14.000000,497.000000,2.000000,23.000000,18.000000,94.000000,112.000000,1.000000,135.000000,325.000000
max,2021.000000,38125.000000,195.000000,1653.000000,129.000000,902.000000,1324.000000,1348.000000,4266.000000,371.000000,4266.000000,4266.000000


In [188]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83414 entries, 0 to 83413
Data columns (total 30 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   dt                        83414 non-null  object 
 1   weekday                   83414 non-null  object 
 2   year                      83414 non-null  int64  
 3   id_driver                 83414 non-null  int64  
 4   id_carrier_number         83414 non-null  object 
 5   dim_carrier_type          83414 non-null  object 
 6   dim_carrier_company_name  83365 non-null  object 
 7   home_base_city            83369 non-null  object 
 8   home_base_state           83369 non-null  object 
 9   carrier_trucks            83414 non-null  object 
 10  num_trucks                83344 non-null  float64
 11  interested_in_drayage     83414 non-null  object 
 12  port_qualified            83414 non-null  object 
 13  signup_source             83414 non-null  object 
 14  ts_sig

# 1. Generate Labels

In [189]:
# converts date from csv to a python datetime object making it easier to work with
def convert_dates(df):
    dates_columns = ['most_recent_load_date', 'first_load_date', 'load_day', 'dt']
    for col_name in dates_columns:
        try:
            df[col_name] = pd.to_datetime(df[col_name], format='%Y-%m-%d')
        except:
            continue

convert_dates(df)
convert_dates(df_score)

In [190]:
df['most_recent_load_date'].head()

0   2021-02-17
1   2021-02-03
2   2020-09-25
3   2021-02-17
4   2017-10-11
Name: most_recent_load_date, dtype: datetime64[ns]

In [191]:
total_loads75 = df.total_loads.quantile(0.75) # finds 75th percentile of loads
most_recent_load_date75 = df.most_recent_load_date.quantile(0.75) # finds 75th percentile of most recent load date

print(total_loads75)
print(most_recent_load_date75)


# Manual Check
# sorted_dts = sorted(list(df.most_recent_load_date))
# quartile_estimate_index = int(len(sorted_dts)*0.75)
# print("SORTED INDEX", sorted_dts[quartile_estimate_index])

325.0
2021-02-14 00:00:00


In [192]:
# new_labels = {"label": {}}
num_days_worked_dict = {}

for index, row in df.iterrows(): # changes the labels in the label columns
    # checks if the load and most recent load date are in the 75th percentile
    if row["total_loads"] >= total_loads75 and row["most_recent_load_date"] >= most_recent_load_date75:
        df.at[index, "label"] = 1
    else:
        df.at[index, "label"] = 0

In [193]:
uniqueValues, occurCount = np.unique(np.array(df["label"]), return_counts=True)
print("Unique Values : " , uniqueValues)
print("Occurrence Count : ", occurCount)

Unique Values :  [0. 1.]
Occurrence Count :  [73021 10393]


In [194]:
loc_arr = []
for index, row in df.iterrows():
    if pd.isnull(row["home_base_city"]) != True and pd.isnull(row["home_base_state"]) != True:
        if (row["home_base_city"], row["home_base_state"]) not in loc_arr:
            loc_arr.append((row["home_base_city"], row["home_base_state"]))
            
# for index, row in df.iterrows():
#     try:
#         x = len(row["home_base_city"])
#         x = len(row["home_base_state"])
#     except:
#         index_num = random.randint(0, len(loc_arr)-1)
#         df.at[index, "home_base_city"] = loc_arr[index_num][0]
#         df.at[index, "home_base_state"] = loc_arr[index_num][1]

In [195]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83414 entries, 0 to 83413
Data columns (total 31 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   dt                        83414 non-null  datetime64[ns]
 1   weekday                   83414 non-null  object        
 2   year                      83414 non-null  int64         
 3   id_driver                 83414 non-null  int64         
 4   id_carrier_number         83414 non-null  object        
 5   dim_carrier_type          83414 non-null  object        
 6   dim_carrier_company_name  83365 non-null  object        
 7   home_base_city            83369 non-null  object        
 8   home_base_state           83369 non-null  object        
 9   carrier_trucks            83414 non-null  object        
 10  num_trucks                83344 non-null  float64       
 11  interested_in_drayage     83414 non-null  object        
 12  port_qualified    

In [196]:
groups = df.groupby("id_driver")

In [197]:
# NO: dt, weekday, year, id_carrier_number, dim_preferred_lanes, load_day, loads
new_arr = []
for key, group in groups:
    group.sort_values(by="load_day", ascending=False, inplace=True)
    temp_arr = []
    temp_arr.append(key)
    
    if group["dim_carrier_type"].nunique() == 2:
        temp_arr.append("Both")
    elif group["dim_carrier_type"].nunique() == 0:
        temp_arr.append(None)
    else:
        temp_arr.append((group["dim_carrier_type"].iloc[0]))
    
    
    idxmax_cols = ["dim_carrier_company_name", 
                   "carrier_trucks", "signup_source", "ts_signup", "ts_first_approved",
                  "days_signup_to_approval"] #"home_base_city", "home_base_state",
    
    for col in idxmax_cols:
        try:
            temp_arr.append(group[col].value_counts().dropna(how="any").idxmax())
        except:
            
            temp_arr.append(None)
    
    try:
        x = len(group["home_base_city"].dropna(how="any").iloc[0])
        x = len(group["home_base_state"].dropna(how="any").iloc[0])
        temp_arr.append(group["home_base_city"].dropna(how="any").iloc[0])
        temp_arr.append(group["home_base_state"].dropna(how="any").iloc[0])
    except:
        index_num = random.randint(0, len(loc_arr)-1)
        temp_arr.append(loc_arr[index_num][0])
        temp_arr.append(loc_arr[index_num][1])
        
    
    
    try:
        temp_arr.append(group["num_trucks"].dropna(how="any").mean())
    except:
        temp_arr.append(None)
        
    iloc_cols = ["interested_in_drayage", "port_qualified", "driver_with_twic", 
                 "first_load_date", "most_recent_load_date", "marketplace_loads_otr", 
                 "marketplace_loads_atlas", "marketplace_loads", "brokerage_loads_otr",
                 "brokerage_loads_atlas", "brokerage_loads", "total_loads"]
    for col in iloc_cols:
        try:
            temp_arr.append(group[col].dropna(how="any").iloc[0])
        except:
            temp_arr.append(None)
    
    temp_arr.append(group.shape[0])
    
    temp_arr.append(((pd.to_datetime(date.today()) - group["most_recent_load_date"].dropna(how="any").iloc[0]).days) * (-1))
    
    temp_arr.append(group["label"].value_counts().dropna(how="any").idxmax())
    
    new_arr.append(np.array(temp_arr))

In [198]:
column_names = ["id_driver", "dim_carrier_type", "dim_carrier_company_name", 
                "carrier_trucks", "signup_source", "ts_signup", "ts_first_approved",
                "days_signup_to_approval", "home_base_city", "home_base_state", "num_trucks", 
                "interested_in_drayage", "port_qualified", "driver_with_twic", 
                "first_load_date", "most_recent_load_date", "marketplace_loads_otr", 
                "marketplace_loads_atlas", "marketplace_loads", "brokerage_loads_otr",
                "brokerage_loads_atlas", "brokerage_loads", "total_loads", "num_trips_made", 
                "days_since_last_load", "label"]

df = pd.DataFrame(np.array(new_arr), columns=column_names)

In [199]:
# convert = ["id_driver", "days_signup_to_approval", "marketplace_loads_otr", 
#                "marketplace_loads_atlas", "marketplace_loads", "brokerage_loads_otr",
#                "brokerage_loads_atlas", "brokerage_loads", "total_loads", "num_trips_made", "label",
#                "num_trucks", "dim_carrier_type", "dim_carrier_company_name", "home_base_city", "home_base_state",
#                "interested_in_drayage", "port_qualified", "signup_source", "driver_with_twic"]
for col in column_names:
    try:
        df[col] = df[col].convert_dtypes()
    except:
        continue

In [200]:
# # NO: dt, weekday, year, id_carrier_number, dim_preferred_lanes, load_day, loads
new_dict = {}
label_dict = {}
most_recent_date_arr = []
num_trips_arr = []
for key, group in groups:
    group.sort_values(by="load_day", ascending=False, inplace=True)
    if key not in new_dict:
        try:
            new_dict[key] = (group["load_day"].dropna(how="any").iloc[0], group.shape[0])
        except:
            new_dict[key] = None
        
        try:
            label_dict[key] = group["label"].dropna(how="any").iloc[0]
        except:
            label_dict[key] = 1

# for index, row in df.iterrows():
#     most_recent_date_arr.append(new_dict[row["id_driver"]][0])
#     num_trips_arr.append(new_dict[row["id_driver"]][1])
# df["num_trips_made"] = np.array(num_trips_arr)

In [201]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5291 entries, 0 to 5290
Data columns (total 26 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   id_driver                 5291 non-null   Int64         
 1   dim_carrier_type          5291 non-null   string        
 2   dim_carrier_company_name  5284 non-null   string        
 3   carrier_trucks            5291 non-null   string        
 4   signup_source             5291 non-null   string        
 5   ts_signup                 5291 non-null   string        
 6   ts_first_approved         3962 non-null   string        
 7   days_signup_to_approval   3962 non-null   Int64         
 8   home_base_city            5291 non-null   string        
 9   home_base_state           5291 non-null   string        
 10  num_trucks                5249 non-null   float64       
 11  interested_in_drayage     5291 non-null   string        
 12  port_qualified      

In [202]:
df["label"].describe()

count    5291.000000
mean        0.010395
std         0.101434
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: label, dtype: float64

In [203]:
groups = df_score.groupby("id_driver")

In [204]:
# # NO: dt, weekday, year, id_carrier_number, dim_preferred_lanes, load_day, loads
# new_arr = []
# for key, group in groups:
#     group.sort_values(by="load_day", ascending=False, inplace=True)
#     temp_arr = []
#     temp_arr.append(key)
    
#     if group["dim_carrier_type"].nunique() == 2:
#         temp_arr.append("Both")
#     elif group["dim_carrier_type"].nunique() == 0:
#         temp_arr.append(None)
#     else:
#         temp_arr.append((group["dim_carrier_type"].iloc[0]))
    
    
#     idxmax_cols = ["dim_carrier_company_name", 
#                    "carrier_trucks", "signup_source", "ts_signup", "ts_first_approved",
#                   "days_signup_to_approval"]
    
#     for col in idxmax_cols:
#         try:
#             temp_arr.append(group[col].value_counts().dropna(how="any").idxmax())
#         except:
#             temp_arr.append(None)
            
    
#     try:
#         x = len(group["home_base_city"].dropna(how="any").iloc[0])
#         x = len(group["home_base_state"].dropna(how="any").iloc[0])
#         temp_arr.append(group["home_base_city"].dropna(how="any").iloc[0])
#         temp_arr.append(group["home_base_state"].dropna(how="any").iloc[0])
#     except:
#         index_num = random.randint(0, len(loc_arr)-1)
#         temp_arr.append(loc_arr[index_num][0])
#         temp_arr.append(loc_arr[index_num][1])
    
    
#     try:
#         temp_arr.append(group["num_trucks"].dropna(how="any").mean())
#     except:
#         temp_arr.append(None)
        
#     iloc_cols = ["interested_in_drayage", "port_qualified", "driver_with_twic", 
#                  "first_load_date", "load_day", "marketplace_loads_otr", 
#                  "marketplace_loads_atlas", "marketplace_loads", "brokerage_loads_otr",
#                  "brokerage_loads_atlas", "brokerage_loads"]
#     for col in iloc_cols:
#         try:
#             temp_arr.append(group[col].dropna(how="any").iloc[0])
#         except:
#             temp_arr.append(None)
            
#     temp_arr.append(group.shape[0])
    
#     temp_arr.append(((pd.to_datetime(date.today()) - group["load_day"].dropna(how="any").iloc[0]).days) * (-1))
    
#     new_arr.append(np.array(temp_arr))

In [205]:
# column_names = ["id_driver", "dim_carrier_type", "dim_carrier_company_name", 
#                 "carrier_trucks", "signup_source", "ts_signup", "ts_first_approved",
#                 "days_signup_to_approval", "home_base_city", "home_base_state", "num_trucks", 
#                 "interested_in_drayage", "port_qualified", "driver_with_twic", 
#                 "first_load_date", "most_recent_load_date", "marketplace_loads_otr", 
#                 "marketplace_loads_atlas", "marketplace_loads", "brokerage_loads_otr",
#                 "brokerage_loads_atlas", "brokerage_loads", "num_trips_made", 
#                 "days_since_last_load"]

# df_score = pd.DataFrame(np.array(new_arr), columns=column_names)

In [206]:
# convert = ["id_driver", "days_signup_to_approval", "marketplace_loads_otr", 
#                "marketplace_loads_atlas", "marketplace_loads", "brokerage_loads_otr",
#                "brokerage_loads_atlas", "brokerage_loads", "num_trips_made",
#                "num_trucks", "dim_carrier_type", "dim_carrier_company_name", "home_base_city", "home_base_state",
#                "interested_in_drayage", "port_qualified", "signup_source", "driver_with_twic"]
# for col in column_names:
#     try:
#         df_score[col] = df_score[col].convert_dtypes()
#     except:
#         continue

In [207]:
# # NO: dt, weekday, year, id_carrier_number, dim_preferred_lanes, load_day, loads
new_dict = {}
most_recent_date_arr = []
num_trips_arr = []
score_labels = []
days_since_last_work_arr = []
trips_dict = {}
for key, group in groups:
    group.sort_values(by="load_day", ascending=False, inplace=True)
    if key not in new_dict:
        try:
            new_dict[key] = (group["load_day"].dropna(how="any").iloc[0], group.shape[0])
        except:
            new_dict[key] = None
    if key not in trips_dict:
        trips_dict[key] = group.shape[0]

for index, row in df_score.iterrows():
    most_recent_date_arr.append(new_dict[row["id_driver"]][0])
    num_trips_arr.append(new_dict[row["id_driver"]][1])
    if row["id_driver"] in label_dict:
        score_labels.append(label_dict[row["id_driver"]])
    else:
        print("Not here: ", row["id_driver"])
        rand_num = random.randint(0, 1000)
        if rand_num <= 125:
            score_labels.append(1)
            label_dict[row["id_driver"]] = 1
        else:
            score_labels.append(0)
            label_dict[row["id_driver"]] = 0
            
    try:
        x = len(row["home_base_city"])
        x = len(row["home_base_state"])
    except:
        index_num = random.randint(0, len(loc_arr)-1)
        df_score.at[index, "home_base_city"] = loc_arr[index_num][0]
        df_score.at[index, "home_base_state"] = loc_arr[index_num][1]
        
    days_since_last_work_arr.append(((pd.to_datetime(date.today()) - row["load_day"]).days) * (-1))

score_labels = np.array(score_labels)
df_score["most_recent_load_date"] = np.array(most_recent_date_arr)
df_score["num_trips_made"] = np.array(num_trips_arr)
df_score["days_since_last_load"] = np.array(days_since_last_work_arr)

Not here:  13711
Not here:  31557
Not here:  16642
Not here:  13761
Not here:  8205
Not here:  35572
Not here:  7623
Not here:  31873
Not here:  12429
Not here:  1288
Not here:  29653
Not here:  20357
Not here:  15073
Not here:  29070
Not here:  10359
Not here:  11514
Not here:  35563
Not here:  7594
Not here:  26545


In [208]:
column_names = ["id_driver", "dim_carrier_type", "dim_carrier_company_name", 
                "carrier_trucks", "signup_source", "ts_signup", "ts_first_approved",
                "days_signup_to_approval", "home_base_city", "home_base_state", "num_trucks", 
                "interested_in_drayage", "port_qualified", "driver_with_twic", 
                "first_load_date", "most_recent_load_date", "marketplace_loads_otr", 
                "marketplace_loads_atlas", "marketplace_loads", "brokerage_loads_otr",
                "brokerage_loads_atlas", "brokerage_loads", "num_trips_made", 
                "days_since_last_load"]

df_temp = df_score.drop(columns = column_names)
to_drop = list(df_temp.columns.values)
df_score = df_score.drop(columns=to_drop)
df_score.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   id_driver                 1000 non-null   int64         
 1   dim_carrier_type          1000 non-null   object        
 2   dim_carrier_company_name  1000 non-null   object        
 3   home_base_city            1000 non-null   object        
 4   home_base_state           1000 non-null   object        
 5   carrier_trucks            1000 non-null   object        
 6   num_trucks                1000 non-null   float64       
 7   interested_in_drayage     1000 non-null   object        
 8   port_qualified            1000 non-null   object        
 9   signup_source             1000 non-null   object        
 10  ts_signup                 1000 non-null   object        
 11  ts_first_approved         854 non-null    object        
 12  days_signup_to_approv

In [209]:
for col in column_names:
    try:
        df_score[col] = df_score[col].convert_dtypes()
    except:
        continue

In [210]:
display(df_score["num_trips_made"])

0      4
1      1
2      1
3      3
4      1
      ..
995    1
996    3
997    1
998    1
999    1
Name: num_trips_made, Length: 1000, dtype: Int64

In [211]:
df_score.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   id_driver                 1000 non-null   Int64         
 1   dim_carrier_type          1000 non-null   string        
 2   dim_carrier_company_name  1000 non-null   string        
 3   home_base_city            1000 non-null   string        
 4   home_base_state           1000 non-null   string        
 5   carrier_trucks            1000 non-null   string        
 6   num_trucks                1000 non-null   Int64         
 7   interested_in_drayage     1000 non-null   string        
 8   port_qualified            1000 non-null   string        
 9   signup_source             1000 non-null   string        
 10  ts_signup                 1000 non-null   string        
 11  ts_first_approved         854 non-null    string        
 12  days_signup_to_approv

In [212]:
# # NO: dt, weekday, year, id_carrier_number, dim_preferred_lanes, load_day, loads



# ["id_driver", "dim_carrier_type", "dim_carrier_company_name", "home_base_city", "home_base_state", 
#                 "carrier_trucks", "signup_source", "ts_signup", "ts_first_approved",
#                 "days_signup_to_approval", "num_trucks", "interested_in_drayage", 
#                 "port_qualified", "driver_with_twic", 
#                 "first_load_date", "most_recent_load_date", "marketplace_loads_otr", 
#                 "marketplace_loads_atlas", "marketplace_loads", "brokerage_loads_otr",
#                 "brokerage_loads_atlas", "brokerage_loads", "total_loads", "num_trips_made", "label"]

drop_cols = ["dt", "weekday", "year", "id_carrier_number", "dim_preferred_lanes", "load_day", "loads"]
for col in drop_cols:
    try:
        df_score = df_score.drop(columns=[col])
    except:
        continue

for col in drop_cols:
    try:
        df = df.drop(columns=[col])
    except:
        continue

In [213]:
df_score.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   id_driver                 1000 non-null   Int64         
 1   dim_carrier_type          1000 non-null   string        
 2   dim_carrier_company_name  1000 non-null   string        
 3   home_base_city            1000 non-null   string        
 4   home_base_state           1000 non-null   string        
 5   carrier_trucks            1000 non-null   string        
 6   num_trucks                1000 non-null   Int64         
 7   interested_in_drayage     1000 non-null   string        
 8   port_qualified            1000 non-null   string        
 9   signup_source             1000 non-null   string        
 10  ts_signup                 1000 non-null   string        
 11  ts_first_approved         854 non-null    string        
 12  days_signup_to_approv

# 3. Basic Statistics

In [214]:
corr_matrix = df.corr()
for col_name in (list(df.columns.values)): # prints all the correlation matrices corresponding to each feature
    try:
        print(col_name)
        display(corr_matrix[col_name].sort_values(ascending=False))
        print('---------------------------------------------------------------------')
    except:
        print("{} is not of type integer".format(col_name))
        print('---------------------------------------------------------------------')

id_driver


id_driver                  1.000000
days_since_last_load       0.839446
num_trucks                 0.169567
brokerage_loads_atlas      0.145307
marketplace_loads_atlas    0.138116
marketplace_loads          0.123344
label                      0.075683
total_loads                0.052722
num_trips_made             0.018007
marketplace_loads_otr     -0.012441
brokerage_loads           -0.037352
brokerage_loads_otr       -0.063795
days_signup_to_approval   -0.734778
Name: id_driver, dtype: float64

---------------------------------------------------------------------
dim_carrier_type
dim_carrier_type is not of type integer
---------------------------------------------------------------------
dim_carrier_company_name
dim_carrier_company_name is not of type integer
---------------------------------------------------------------------
carrier_trucks
carrier_trucks is not of type integer
---------------------------------------------------------------------
signup_source
signup_source is not of type integer
---------------------------------------------------------------------
ts_signup
ts_signup is not of type integer
---------------------------------------------------------------------
ts_first_approved
ts_first_approved is not of type integer
---------------------------------------------------------------------
days_signup_to_approval


days_signup_to_approval    1.000000
brokerage_loads_otr        0.048116
brokerage_loads            0.027929
num_trips_made             0.010462
marketplace_loads_otr      0.007436
total_loads               -0.043765
num_trucks                -0.053498
label                     -0.064084
marketplace_loads         -0.089268
brokerage_loads_atlas     -0.092088
marketplace_loads_atlas   -0.099058
days_since_last_load      -0.628732
id_driver                 -0.734778
Name: days_signup_to_approval, dtype: float64

---------------------------------------------------------------------
home_base_city
home_base_city is not of type integer
---------------------------------------------------------------------
home_base_state
home_base_state is not of type integer
---------------------------------------------------------------------
num_trucks


...

---------------------------------------------------------------------
days_since_last_load


days_since_last_load       1.000000
id_driver                  0.839446
num_trips_made             0.284921
marketplace_loads          0.236472
total_loads                0.216076
marketplace_loads_atlas    0.202243
brokerage_loads_atlas      0.179735
num_trucks                 0.163449
marketplace_loads_otr      0.157085
label                      0.142288
brokerage_loads            0.092076
brokerage_loads_otr        0.061265
days_signup_to_approval   -0.628732
Name: days_since_last_load, dtype: float64

---------------------------------------------------------------------
label


label                      1.000000
marketplace_loads          0.584442
marketplace_loads_atlas    0.577596
total_loads                0.541811
num_trips_made             0.481593
brokerage_loads_atlas      0.327819
brokerage_loads            0.293505
brokerage_loads_otr        0.239032
marketplace_loads_otr      0.163324
days_since_last_load       0.142288
id_driver                  0.075683
num_trucks                -0.035292
days_signup_to_approval   -0.064084
Name: label, dtype: float64

---------------------------------------------------------------------


 Also year and TODO_FIND_COLUMN_NAME_2 and year are highly correlated and have a similar impact on label, so we could drop one? 

Is there really a need for brokerage_loads when it is so highly correlated to brokerage_loads_otr due to the vast majority of shipments being delivered over-the-road as compared to via ATLAS? 

I have the same question about total_loads due to the vast majority of loads being brokerage loads...

What's the point of having both year and date?

We can remove the id_carrier_number column from this dataset as it is not relevant to predicting a label of 0 or 1 (When trying to find high performing drivers, we need to know their carrier number, so we can extract the id_carrier_number column for now...)

We could one-hot-encode sign-up source and see its effect on labels.

We can remove the ts_first_approved column because the date of approval shouldn't matter that much but instead the days_signup_to_approval matter.

dim_preferred_lanes only has a few values so we can either remove the column or impute values.

Also first_load_date, most_recent_load_date and load_day shouldn't matter much. Instead we can have values such as: number of days doing the job = most_recent_load_date - first_load_date
AND
days_from_last_load_to_today = todays_date - most_recent_load_date

There are also a couple other features we need to impute.

Also, only people that are port qualified can provide drayage services, so we should create a field called qualified_and_interest_in_drayage which is only 1 (yes) when interested_in_drayage = "yes" and port_qualified = "yes". We can also cross these features...

# 4. Data Feature Extraction Plan and Pipeline

In [215]:
# df["location"] = list(zip(df["home_base_city"], df["home_base_state"]))# feature cross to get (city, state) tuple
# # feature cross for interested in drayage and port qualified
# df["drayage_interested_port_qualified"] = list(zip(df["interested_in_drayage"], df["port_qualified"]))
# display(df["location"])
# display(df["drayage_interested_port_qualified"])

In [216]:
def drayage_feature_cross(df):
    loc_cross = list(zip(df["home_base_city"], df["home_base_state"]))# feature cross to get (city, state) tuple
    # feature cross for interested in drayage and port qualified
    drayage_cross = list(zip(df["interested_in_drayage"], df["port_qualified"]))

    drayage_arr = []
    for list_item in drayage_cross:
        if list_item[0] == "yes" and list_item[1] == "yes":
            drayage_arr.append("000001")
        if list_item[0] == "yes" and list_item[1] == "no":
            drayage_arr.append("000010")
        if list_item[0] == "no" and list_item[1] == "yes":
            drayage_arr.append("000100")
        if list_item[0] == "no" and list_item[1] == "no":
            drayage_arr.append("001000")
        if list_item[0] == "not specified" and list_item[1] == "yes":
            drayage_arr.append("010000")
        if list_item[0] == "not specified" and list_item[1] == "no":
            drayage_arr.append("100000")

    df["drayage_interested_port_qualified"] = np.array(drayage_arr)
    display(df["drayage_interested_port_qualified"])

drayage_feature_cross(df)
drayage_feature_cross(df_score)

0       100000
1       100000
2       100000
3       100000
4       100000
         ...  
5286    100000
5287    000001
5288    000001
5289    000001
5290    000001
Name: drayage_interested_port_qualified, Length: 5291, dtype: object

0      100000
1      000010
2      100000
3      000010
4      000010
        ...  
995    100000
996    100000
997    100000
998    100000
999    000010
Name: drayage_interested_port_qualified, Length: 1000, dtype: object

In [217]:
id_driver_number_col = np.array(df["id_driver"]) # extract id_driver column
id_driver_number_col_score = np.array(df_score["id_driver"]) # extract id_driver column

drop_cols = ["id_driver", "interested_in_drayage", "port_qualified", 
             "ts_signup", "ts_first_approved"] #"home_base_city","home_base_state",
for col in drop_cols:
    try:
        df_score = df_score.drop(columns=[col])
    except:
        continue

for col in drop_cols:
    try:
        df = df.drop(columns=[col])
    except:
        continue

In [218]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5291 entries, 0 to 5290
Data columns (total 22 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   dim_carrier_type                   5291 non-null   string        
 1   dim_carrier_company_name           5284 non-null   string        
 2   carrier_trucks                     5291 non-null   string        
 3   signup_source                      5291 non-null   string        
 4   days_signup_to_approval            3962 non-null   Int64         
 5   home_base_city                     5291 non-null   string        
 6   home_base_state                    5291 non-null   string        
 7   num_trucks                         5249 non-null   float64       
 8   driver_with_twic                   5291 non-null   string        
 9   first_load_date                    5291 non-null   datetime64[ns]
 10  most_recent_load_date              5

In [219]:
names = {}
for index, row in df.iterrows():
    names[row["dim_carrier_company_name"]] = int(names.get(row["dim_carrier_company_name"], 0) + 1)
listo = list(names.items())
listo.sort(reverse=True, key=lambda x: int(x[1]))
#listo[:50]

In [220]:
listo = listo[:50]
count_50 = sum([x[1] for x in listo])
print("# 50: ", count_50)
print("Percentage 50: ", count_50/len(names))

names_arr = [tuples[0] for tuples in listo]
print(names_arr)

# 50:  1290
Percentage 50:  0.5176565008025682
['NFS asset Drayage', 'MC Express Trucking LLC', 'Roadrunner Transportation', 'Dong Fang Marketing Inc', 'ROADMOND LOGISTICS INC.', 'Consistent Trucking Inc', 'BLUE FREIGHT TRANSPORT INC', 'Convoy Express', 'Mega Fleet', 'iDC Drayage', 'Cross World Logistics', 'USA Diamonds Trucking', 'pointdirect', 'Chaidez Trucking', 'Saia LTL Freight', 'American Better Choice Corporation', 'American Freightways Lp.', 'Fastrucking', 'MERIDIAN LOGISTICS INC', 'Carlos Flores', 'J&G Transportation Group Inc', 'KLF transport inc', 'Star Rain LLC', 'Starco Logistics Inc.', '664 Transport', 'FTS EXPRESS INC', 'IDC OTR', "Luna's Transportation group", 'AGRAMONT TRANSPORT INC', 'BGood VirtueT Inc', 'Geber Freight', 'R&Y Castellanos Trucking Inc.', 'JC Transport', 'Great Qin Transportation LLC', 'JM Express Inc', 'MT Brothers Groups', 'cbt trucking', 'Road Eagle Logistics Corp', 'O.A. EXPRESS INC', 'AMPAK Logistics INC.', 'Kuang Trucking Inc.', 'nolan transportat

In [221]:
def bucketize(df):
    days_worked = []
    for index, row in df.iterrows(): # bucketize the most frequent dim_carrier_company names, 
                                     # put less frequent names in a single bucket
        try:
            if row["dim_carrier_company_name"] not in names_arr:
                df.at[index, "dim_carrier_company_name"] = "Other"
        except:
            df.at[index, "dim_carrier_company_name"] = "Other"

        # find number of days driver has worked
        if row["most_recent_load_date"] != np.nan and row["first_load_date"] != np.nan:
            days_worked.append((row["most_recent_load_date"] - row["first_load_date"]).days)
        else:
            days_worked.append(None)
    df["days_tenured"] = np.array(days_worked)

bucketize(df)
bucketize(df_score)

In [222]:
# # Get total_loads for df_score in order to find labels
# total_loads = []
# for index, row in df_score.iterrows():
#     total_loads.append(row["marketplace_loads"] + row["brokerage_loads"])
# df_score["total_loads"] = np.array(total_loads)

In [223]:
# total_loads_score_75 = df_score.total_loads.quantile(0.75) # finds 75th percentile of loads
# most_recent_load_date_score_75 = df_score.most_recent_load_date.quantile(0.75) # finds 75th percentile of most recent load date

# print(total_loads_score_75)
# print(most_recent_load_date_score_75)

In [224]:
# score_labels = []
# for index, row in df_score.iterrows(): # changes the labels in the label columns
#     # checks if the load and most recent load date are in the 75th percentile
#     if row["total_loads"] >= total_loads_score_75 and row["most_recent_load_date"] >= most_recent_load_date_score_75:
#         score_labels.append(1)
#     else:
#         score_labels.append(0)
# print(len(score_labels))
# print(score_labels)

In [225]:
drop_cols = ["most_recent_load_date", "first_load_date", "weekday", "load_day", "total_loads"]
for col in drop_cols:
    try:
        df_score = df_score.drop(columns=[col])
    except:
        continue

for col in drop_cols:
    try:
        df = df.drop(columns=[col])
    except:
        continue

In [226]:
df_unlabeled = df.drop(columns=["label"])
labels = df["label"].copy()

In [227]:
uniqueValues, occurCount = np.unique(labels, return_counts=True)
print("Unique Values : " , uniqueValues)
print("Occurrence Count : ", occurCount)

Unique Values :  [0 1]
Occurrence Count :  [5236   55]


In [228]:
df_unlabeled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5291 entries, 0 to 5290
Data columns (total 19 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   dim_carrier_type                   5291 non-null   string 
 1   dim_carrier_company_name           5291 non-null   string 
 2   carrier_trucks                     5291 non-null   string 
 3   signup_source                      5291 non-null   string 
 4   days_signup_to_approval            3962 non-null   Int64  
 5   home_base_city                     5291 non-null   string 
 6   home_base_state                    5291 non-null   string 
 7   num_trucks                         5249 non-null   float64
 8   driver_with_twic                   5291 non-null   string 
 9   marketplace_loads_otr              5291 non-null   Int64  
 10  marketplace_loads_atlas            5291 non-null   Int64  
 11  marketplace_loads                  5291 non-null   Int64

In [229]:
df_score.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 19 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   dim_carrier_type                   1000 non-null   string
 1   dim_carrier_company_name           1000 non-null   string
 2   home_base_city                     1000 non-null   string
 3   home_base_state                    1000 non-null   string
 4   carrier_trucks                     1000 non-null   string
 5   num_trucks                         1000 non-null   Int64 
 6   signup_source                      1000 non-null   string
 7   days_signup_to_approval            854 non-null    Int64 
 8   driver_with_twic                   1000 non-null   string
 9   marketplace_loads_otr              1000 non-null   Int64 
 10  marketplace_loads_atlas            1000 non-null   Int64 
 11  marketplace_loads                  1000 non-null   Int64 
 12  brokera

In [230]:
cols = list(df_unlabeled.columns.values)
print(cols)

['dim_carrier_type', 'dim_carrier_company_name', 'carrier_trucks', 'signup_source', 'days_signup_to_approval', 'home_base_city', 'home_base_state', 'num_trucks', 'driver_with_twic', 'marketplace_loads_otr', 'marketplace_loads_atlas', 'marketplace_loads', 'brokerage_loads_otr', 'brokerage_loads_atlas', 'brokerage_loads', 'num_trips_made', 'days_since_last_load', 'drayage_interested_port_qualified', 'days_tenured']


In [231]:
df_score = df_score[cols]

df_score['num_trucks'] = pd.to_numeric(df_score['num_trucks'], errors='coerce')
df_score['days_signup_to_approval'] = pd.to_numeric(df_score['days_signup_to_approval'], errors='coerce')

df_unlabeled['num_trucks'] = pd.to_numeric(df_unlabeled['num_trucks'], errors='coerce')
df_unlabeled['days_signup_to_approval'] = pd.to_numeric(df_unlabeled['days_signup_to_approval'], errors='coerce')

# df_unlabeled['home_base_city'].astype(str) #= to_string(df_unlabeled['home_base_city'], errors='coerce')
# df_unlabeled['home_base_state'].astype(str) #= pd.to_string(df_unlabeled['home_base_state'], errors='coerce')

convert = ["id_driver", "days_signup_to_approval", "marketplace_loads_otr", 
               "marketplace_loads_atlas", "marketplace_loads", "brokerage_loads_otr",
               "brokerage_loads_atlas", "brokerage_loads", "num_trips_made",
               "num_trucks", "dim_carrier_type", "dim_carrier_company_name",
               "interested_in_drayage", "port_qualified", "signup_source", "driver_with_twic",
          "home_base_city","home_base_state"] #"home_base_city","home_base_state"
for col in convert:
    try:
        df_score[col] = df_score[col].convert_dtypes()
    except:
        continue

for col in convert:
    try:
        df_unlabeled[col] = df_unlabeled[col].convert_dtypes()
    except:
        continue

In [232]:
df_concat = pd.concat([df_unlabeled, df_score], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6291 entries, 0 to 6290
Data columns (total 19 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   dim_carrier_type                   6291 non-null   string
 1   dim_carrier_company_name           6291 non-null   string
 2   carrier_trucks                     6291 non-null   string
 3   signup_source                      6291 non-null   string
 4   days_signup_to_approval            4816 non-null   Int64 
 5   home_base_city                     6291 non-null   string
 6   home_base_state                    6291 non-null   string
 7   num_trucks                         6249 non-null   object
 8   driver_with_twic                   6291 non-null   string
 9   marketplace_loads_otr              6291 non-null   Int64 
 10  marketplace_loads_atlas            6291 non-null   Int64 
 11  marketplace_loads                  6291 non-null   Int64 
 12  broker

In [233]:
for col in convert:
    try:
        df_concat[col] = df_concat[col].convert_dtypes()
    except:
        continue

In [234]:
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6291 entries, 0 to 6290
Data columns (total 19 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   dim_carrier_type                   6291 non-null   string 
 1   dim_carrier_company_name           6291 non-null   string 
 2   carrier_trucks                     6291 non-null   string 
 3   signup_source                      6291 non-null   string 
 4   days_signup_to_approval            4816 non-null   Int64  
 5   home_base_city                     6291 non-null   string 
 6   home_base_state                    6291 non-null   string 
 7   num_trucks                         6249 non-null   float64
 8   driver_with_twic                   6291 non-null   string 
 9   marketplace_loads_otr              6291 non-null   Int64  
 10  marketplace_loads_atlas            6291 non-null   Int64  
 11  marketplace_loads                  6291 non-null   Int64

In [235]:
# df_concat = df_concat.drop(columns=["marketplace_loads", "brokerage_loads"])

In [236]:
imputer = IterativeImputer()
# df = imputer.fit_transform(df)
categorical_features_one_hot = ["dim_carrier_type", "dim_carrier_company_name", "carrier_trucks", 
                                "signup_source", "driver_with_twic", "home_base_city","home_base_state"] #"home_base_city","home_base_state"

df_num = df_concat.drop(columns=categorical_features_one_hot)
# df_num = df_concat
numerical_features = list(df_num)

num_pipeline = Pipeline([
        ('imputer', imputer),
        ('std_scaler', StandardScaler()),
    ])

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, numerical_features),
        ("cat", OneHotEncoder(sparse=False), categorical_features_one_hot), #sparse=False
    ])
df_prepared = full_pipeline.fit_transform(df_concat)

In [237]:
display(df_prepared)

array([[ 1.85116313, -0.39965744, -0.20109732, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.87262531, -0.39965744, -0.20109732, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.88612658, -0.39965744, -0.20109732, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.07263803, -0.39965744,  0.23165021, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.75297732,  1.95624695, -0.20109732, ...,  0.        ,
         0.        ,  0.        ],
       [-1.00674481, -0.18762604, -0.10999258, ...,  0.        ,
         0.        ,  0.        ]])

In [238]:
# X_train_temp = df_prepared[:5291]
# y_train_temp = labels
# X_test = df_prepared[5291:]
# y_test = score_labels

# y_train_temp = y_train_temp.astype('int')

In [239]:
X = df_prepared[:5291]
y = labels
X_test_score = df_prepared[5291:]
y_test_score = score_labels
y_test_score = np.array(y_test_score)
y = y.astype('int')
X, y = SMOTE().fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [240]:
# X_train, y_train = SMOTE().fit_resample(X_train_temp, y_train_temp)

In [241]:
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)
print(X_test_score.shape, y_test_score.shape)

(8377, 647) (2095, 647)
(8377,) (2095,)
(1000, 647) (1000,)


# 5. Linear Regression

In [242]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
lr_predicted = lin_reg.predict(X_test)

In [243]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_test, lr_predicted)
rmse = np.sqrt(mse)
rmse

1504445460.6053133

In [244]:
from sklearn.metrics import r2_score

r2_score(y_test, lr_predicted)

-9.054928564197502e+18

In [245]:
import statsmodels.api as sm

X_new = sm.add_constant(X_train)
toyregr_sm = sm.OLS(y_train.astype(float), X_new.astype(float))
results_sm = toyregr_sm.fit()

print(results_sm.summary())

                            OLS Regression Results                            
Dep. Variable:                  label   R-squared:                       0.904
Model:                            OLS   Adj. R-squared:                  0.897
Method:                 Least Squares   F-statistic:                     137.8
Date:                Thu, 18 Mar 2021   Prob (F-statistic):               0.00
Time:                        12:04:37   Log-Likelihood:                 3721.5
No. Observations:                8377   AIC:                            -6373.
Df Residuals:                    7842   BIC:                            -2610.
Df Model:                         534                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0064      0.025     -0.258      0.7

In [246]:
from sklearn.linear_model import LogisticRegression

# y_temp = y_train.astype('int')

log_reg = LogisticRegression(max_iter=1000, multi_class='ovr')
log_reg.fit(X_train, y_train)
log_predicted = log_reg.predict(X_test)

In [247]:
mse = mean_squared_error(y_test, log_predicted)
rmse = np.sqrt(mse)
rmse

0.06908885358534668

In [248]:
r2_score(y_test, log_predicted)

0.9809037494280239

In [249]:
from sklearn.metrics import confusion_matrix

accuracy = (accuracy_score(y_test, log_predicted))
print("accuracy = {}%".format(accuracy))
print("roc_auc_score = {}".format(roc_auc_score(y_test, log_predicted)))
print("F1-score = {}".format(f1_score(y_test, log_predicted)))

confusion_matrix(y_test, log_predicted)

accuracy = 0.9952267303102625%
roc_auc_score = 0.9952874646559849
F1-score = 0.9951876804619826


array([[1051,   10],
       [   0, 1034]])

# 6. PCA

In [250]:
pca = PCA(n_components=0.95) # Create an instance of PCA model
pca.fit(X_train) # Fit X_train to PCA
X_train = pca.transform(X_train) # transform training data
X_test = pca.transform(X_test) # transform test data
X_test_score = pca.transform(X_test_score) # transform score test data
print(pca.explained_variance_)
print(pca.n_components_)

[21.54605377 14.82162681  4.7692357   3.41489078  1.33804063  0.9145885
  0.65036437  0.4578647 ]
8


In [251]:
print(X_train)

[[-4.49171946e+00 -3.36971281e-01 -6.17426433e-02 ...  3.96689156e-01
  -4.78261064e-01  1.97966321e-01]
 [ 1.96275186e+00 -1.62008004e+00 -4.61421032e-01 ... -6.76517519e-01
  -6.82638499e-01  1.34512634e-01]
 [ 6.17950696e-01 -2.08098660e+00  3.24847168e-03 ...  1.17915633e-01
  -8.82436980e-02 -2.37155661e-01]
 ...
 [ 2.71412734e+00 -2.04138671e+00 -1.08705756e+00 ... -6.91001739e-01
  -2.58973097e-01 -1.83220952e-01]
 [-4.09048940e+00  4.01143639e-01 -2.65999648e-01 ...  2.06503507e-01
  -2.76134393e-02  1.71059491e-02]
 [ 1.14684492e+01 -5.10986595e+00 -2.09034666e+00 ... -2.78057838e-01
   1.51084008e-01  1.10663819e-01]]


In [252]:
print(X_test)

[[ 1.47201017  5.85476642 -1.33748134 ... -2.309122   -0.21649719
   0.10033926]
 [ 3.90933966 -2.09572577  2.1669793  ... -0.1756743  -0.24712945
   0.48814435]
 [-4.57451127 -0.31212458 -0.0766936  ...  0.37210484  0.08243408
   0.47581007]
 ...
 [-3.45461439  0.93716425 -0.53369296 ... -0.1644751  -0.06562727
  -0.1109223 ]
 [14.73793244 -6.31443182 -2.99549173 ...  0.27162949  0.35510181
   0.18361845]
 [ 2.9264617   4.89898943 -3.12135157 ...  0.1110642  -0.97888076
   0.76024239]]


In [253]:
print(X_test_score)

[[-3.73004388e+00  8.59372492e-01  1.14088028e-01 ...  2.59492874e-01
  -2.01512739e-01  7.07934758e-02]
 [-3.96337938e+00 -3.68476799e-01  1.88952304e-01 ...  6.05698946e-01
   2.19411540e-01 -6.48924524e-01]
 [-3.53296980e+00 -1.52054286e-01 -7.13636505e-01 ...  4.69037752e-01
  -4.18900930e-01  3.54243710e-01]
 ...
 [-4.29130028e+00 -2.91374556e-01 -3.40155628e-01 ...  5.86090380e-01
  -5.82476738e-01  3.03477312e-01]
 [-4.48422097e+00  6.53433125e-01  2.47730787e-03 ... -3.07443324e-01
   2.21822736e+00  1.10978598e+00]
 [-3.71865867e+00 -9.44468529e-03  2.75731540e-01 ...  9.40557431e-03
   2.66634267e-01 -7.36657495e-01]]


In [254]:
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)
print(X_test_score.shape, y_test_score.shape)

(8377, 8) (2095, 8)
(8377,) (2095,)
(1000, 8) (1000,)


# 7. Ensemble

In [ ]:
# y_train = y_train.astype('int')
# y_test = y_test.astype('int')

In [182]:
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import r2_score

# assuming we have X_train,X_test,y_train,y_test at this time
# I first run Random Forest using random hard coded settings to get a baseline
rf = RandomForestClassifier(n_estimators=80,max_depth=7,max_features=3)
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
test_score = r2_score(y_test,y_pred)
print(test_score)
accuracy = (accuracy_score(y_test, y_pred))
print("accuracy = {}%".format(accuracy))
print("roc_auc_score = {}".format(roc_auc_score(y_test, y_pred)))
print("F1-score = {}".format(f1_score(y_test, y_pred)))

confusion_matrix(y_test, y_pred)

0.9828133744852215
accuracy = 0.9957040572792363%
roc_auc_score = 0.9957587181903864
F1-score = 0.9956668271545499


array([[1052,    9],
       [   0, 1034]])

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
# I then use RandomizedSearchCV to find the optimal hyperparameters
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1500, num = 30)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(5, 100, num = 10)]
max_depth.append(None)
min_samples_split = [2, 5, 7, 10]
min_samples_leaf = [2, 5, 7, 10]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

rf = RandomForestClassifier(class_weight='balanced', random_state=42)

rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(X_train, y_train)
rf_random.best_params_

In [ ]:
rf = RandomForestRegressor(n_estimators=411,max_depth=68,max_features='auto',min_samples_split=5,min_samples_leaf=2,bootstrap=True)
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
misclassified = 0
for i in range(len(y_pred)):
    if y_pred[i] != y_test[i]:
        misclassified += 1
accuracy = ((1000-misclassified)*100/len(y_pred))
print("accuracy = {}%".format(accuracy))

In [ ]:
# from sklearn.datasets import make_regression
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import r2_score

# # assuming we have X_train,X_test,y_train,y_test at this time
# # I first run Random Forest using random hard coded settings to get a baseline
# rf = RandomForestRegressor(n_estimators=80,max_depth=7,max_features=3)
# rf.fit(X_train,y_train)
# y_pred = rf.predict(X_test)
# test_score = r2_score(y_test,y_pred)
# test_score

In [ ]:
# from sklearn.model_selection import RandomizedSearchCV
# import numpy as np
# # I then use RandomizedSearchCV to find the optimal hyperparameters
# n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1500, num = 10)]
# max_features = ['auto', 'sqrt']
# max_depth = [int(x) for x in np.linspace(5, 100, num = 10)]
# max_depth.append(None)
# min_samples_split = [2, 5, 7, 10, 20]
# min_samples_leaf = [2, 5, 7, 10]
# bootstrap = [True, False]
# random_grid = {'n_estimators': n_estimators,
#                'max_features': max_features,
#                'max_depth': max_depth,
#                'min_samples_split': min_samples_split,
#                'min_samples_leaf': min_samples_leaf,
#                'bootstrap': bootstrap}

# rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 5, verbose=2, random_state=42, n_jobs = -1)
# rf_random.fit(X_train, y_train)
# rf_random.best_params_

In [ ]:
# # I then output the r2 score again as a sanity check to verify that my RanomdizedSearchCV actually did find the best settings
# rf = RandomForestRegressor(n_estimators=411,max_depth=68,max_features='auto',min_samples_split=5,min_samples_leaf=2,bootstrap=True)
# rf.fit(X_train,y_train)
# y_pred = rf.predict(X_test)
# from sklearn.metrics import r2_score
# test_score = r2_score(y_test,y_pred)
# test_score

In [ ]:
# we then use the hyperparameters we found from the RandomizedSearchCV to do a second more thorough check around that range
from sklearn.model_selection import GridSearchCV
param_grid = {
    'bootstrap': [True],
    'max_depth': [65, 68, 71],
    'max_features': ['auto'],
    'min_samples_leaf': [2, 4, 6],
    'min_samples_split': [3, 5, 7],
    'n_estimators': [375, 411, 500]
}
# Create a based model
rf = RandomForestRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 5, n_jobs = -1, verbose = 2)
grid_search.fit(X_train, y_train)
grid_search.best_params_

In [ ]:
# we then output the results using the optimal hyperparameters to check that our model has improved
rf = RandomForestRegressor(n_estimators=375,max_depth=68,max_features='auto',min_samples_split=3,min_samples_leaf=2,bootstrap=True)
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
from sklearn.metrics import r2_score
test_score = r2_score(y_test,y_pred)
test_score

In [101]:
# AdaBoost using the same settings
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

y_train = y_train.astype('float')

dt = DecisionTreeClassifier(max_depth=10,max_features='auto',min_samples_leaf=2,min_samples_split=3,
                           random_state=42, class_weight='balanced')
ab = AdaBoostClassifier(base_estimator=dt,learning_rate=1,n_estimators=450)
ab.fit(X_train,y_train)
y_pred = ab.predict(X_test)
test_score = r2_score(y_test,y_pred)
accuracy = (accuracy_score(y_test, y_pred))
print("accuracy = {}%".format(accuracy))
print("roc_auc_score = {}".format(roc_auc_score(y_test, y_pred)))
print("F1-score = {}".format(f1_score(y_test, y_pred)))

accuracy = 0.99618138424821%
roc_auc_score = 0.996229971724788
F1-score = 0.9961464354527938


# 8. Neural Network Classifier

In [285]:
# Neural network
def create_model():
    model = Sequential()
    model.add(Dense(16, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(12, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', keras.metrics.Precision(),
                                                                        keras.metrics.Recall(), 
                                                                         keras.metrics.FalsePositives()])
    return model

model = create_model()

model.fit(X_train, y_train, batch_size=20, epochs=50)

y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)
accuracy = (accuracy_score(y_test, y_pred))
print("accuracy = {}%".format(accuracy))
print("roc_auc_score = {}".format(roc_auc_score(y_test, y_pred)))
print("F1-score = {}".format(f1_score(y_test, y_pred)))
confusion_matrix(y_test, y_pred)

Epoch 1/50
419/419 [==============================] - 1s 888us/step - loss: 0.3702 - accuracy: 0.8403 - precision_17: 0.9777 - recall_17: 0.7026 - false_positives_17: 39.3714
Epoch 2/50
419/419 [==============================] - 0s 1ms/step - loss: 0.0508 - accuracy: 0.9875 - precision_17: 0.9801 - recall_17: 0.9956 - false_positives_17: 41.7238
Epoch 3/50
419/419 [==============================] - 0s 988us/step - loss: 0.0334 - accuracy: 0.9928 - precision_17: 0.9858 - recall_17: 1.0000 - false_positives_17: 31.2833
Epoch 4/50
419/419 [==============================] - 0s 961us/step - loss: 0.0301 - accuracy: 0.9932 - precision_17: 0.9870 - recall_17: 1.0000 - false_positives_17: 27.0405
Epoch 5/50
419/419 [==============================] - 0s 983us/step - loss: 0.0266 - accuracy: 0.9938 - precision_17: 0.9876 - recall_17: 1.0000 - false_positives_17: 25.9905
Epoch 6/50
419/419 [==============================] - 0s 1ms/step - loss: 0.0229 - accuracy: 0.9947 - precision_17: 0.9896 - re

KeyboardInterrupt: 

In [279]:
y_pred = model.predict(X_test_score)
y_pred = (y_pred > 0.5)
print(len(y_pred))
accuracy = (accuracy_score(y_test_score, y_pred))
print("accuracy = {}%".format(accuracy))
print("roc_auc_score = {}".format(roc_auc_score(y_test_score, y_pred)))
print("F1-score = {}".format(f1_score(y_test_score, y_pred)))
confusion_matrix(y_test_score, y_pred)

1000
accuracy = 0.89%
roc_auc_score = 0.6222197672705848
F1-score = 0.375


array([[857,  19],
       [ 91,  33]])

# 9. Cross-Validate

In [280]:
pca = PCA(n_components=0.95) # Create an instance of PCA model
pca.fit(X) # Fit X_train to PCA
X_pca = pca.transform(X) # transform training data

In [286]:
from sklearn.model_selection import cross_val_score
from keras.wrappers.scikit_learn import KerasClassifier

kfold = model_selection.KFold(n_splits=10, random_state=42, shuffle=True)

rf_model_kfold = RandomForestClassifier(n_estimators=80,max_depth=7,max_features=3)

rf_results_kfold = model_selection.cross_val_score(rf_model_kfold, X_pca, y, cv=kfold)

print("RF Accuracy: %.2f%%" % (rf_results_kfold.mean()*100.0)) 

nn = KerasClassifier(build_fn=create_model, epochs=50, batch_size=20, verbose=0)

nn_results_kfold = model_selection.cross_val_score(nn, X_pca, y, cv=kfold)

print("NN Accuracy: %.2f%%" % (nn_results_kfold.mean()*100.0))

RF Accuracy: 99.57%
NN Accuracy: 99.72%


# 10. Custom Model

In [283]:
kernel_types = ['linear', 'rbf', 'sigmoid']
for kernel in kernel_types:
    svc = SVC(kernel=kernel, class_weight='balanced', random_state=42, max_iter=100) #class_weight='balanced'
    svc.fit(X_train, y_train)
    y_pred = svc.predict(X_test)
    accuracy = (accuracy_score(y_test, y_pred))
    print("accuracy for kernel={} = {}%".format(kernel, accuracy))
    print("roc_auc_score = {}".format(roc_auc_score(y_test, y_pred)))
    print("F1-score = {}".format(f1_score(y_test, y_pred)))
    display(confusion_matrix(y_test, y_pred))

accuracy for kernel=linear = 0.7718377088305489%
roc_auc_score = 0.769055688130427
F1-score = 0.7053020961775586


/Users/shrenik/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'


array([[1045,   16],
       [ 462,  572]])

accuracy for kernel=rbf = 0.9522673031026253%
roc_auc_score = 0.9519517370751654
F1-score = 0.9504459861248761


/Users/shrenik/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'


array([[1036,   25],
       [  75,  959]])

accuracy for kernel=sigmoid = 0.9608591885441528%
roc_auc_score = 0.9607665481088786
F1-score = 0.9600778967867576


/Users/shrenik/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'


array([[1027,   34],
       [  48,  986]])

In [ ]:
# Neural network
model = Sequential()
model.add(Dense(16, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(12, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', keras.metrics.Precision(),
                                                                    keras.metrics.Recall(), 
                                                                     keras.metrics.FalsePositives()])

model.fit(X_train, y_train, batch_size=20, epochs=100)

y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)
accuracy = (accuracy_score(y_test, y_pred))
print("accuracy = {}%".format(accuracy))
print("roc_auc_score = {}".format(roc_auc_score(y_test, y_pred)))
print("F1-score = {}".format(f1_score(y_test, y_pred)))
confusion_matrix(y_test, y_pred)